# Data Processing script for the NSM/SWEML v2.0
This .ipynb script uses python module for retrieving NASA ASO observations, locating nearest SNOTEL sites, connecting SNOTEL obs with ASO obs, and add geospatial features to the ML training/testing/hindcast dataframes.

# Next steps 

- the SE and SW rockies have the same number of sites, make sure they are not the same...
- process ASO data, e.g. swe_m < 0.1 = 0, convert to cm to be consistent with monitoring sites and traditional mesurement. 
- document scripts
- add new sites (e.g., regionval) to training DF with all the respective spatial resolution information
- add precipitation phase features (seasonal accumulated rain precip, seasonal accumulated snow precip as a function of temperature)
- explore adding other features stemming from SNOTEL, remote sensing (LULC), Snow Classifications (Sturms), energy balance

In [57]:
import os
import model_Domain
HOME = os.path.expanduser('~')

#make SWEMLv2.0 modeling domain for western USA
region_list = model_Domain.modeldomain()
region_list.remove('NorthernRockies') # There is no ASO data for this region
region_list = ['Northwest', 'SouthernRockies', 'Southwest']
region_list

Checking for required files
ground_measures_metadata.parquet is local


['Northwest', 'SouthernRockies', 'Southwest']

In [ ]:
from ASOget import ASODownload, ASODataProcessing

# import earthaccess https://earthaccess.readthedocs.io/en/latest/howto/authenticate/
# earthaccess.login(persist=True)

# Inputs for fetching ASO data for a region
short_name = 'ASO_50M_SWE'
version = '1'
time_start = '2013-04-02T00:00:00Z'
time_end = '2019-07-19T23:59:59Z'
output_res = 300 #desired spatial resoultion in meters (m)
directory = "Raw_ASO_Data"

#Get ASO data
for region in region_list:
    print(region)
    folder_name = f"{region}/{directory}"
    data_tool = ASODownload(short_name, version)
    b_box = data_tool.BoundingBox(region)  
    url_list = data_tool.cmr_search(time_start, time_end, region, b_box)
    data_tool.cmr_download(directory, region)

    #Convert ASO tifs to parquet
    data_processor = ASODataProcessing()
    data_processor.convert_tiff_to_parquet_multiprocess(folder_name, output_res, region) 

## Get Snotel and CDEC in situ observations
- clean in situ observations, specifically the CDEC sites, need a data processing method to remove outtliers and nan/0 obs
- Ideas - add nearest sites elevation, distance from cell, then can bypass sites with bad data. 

In [ ]:
# Get in situ observations
import get_InSitu_obs
import numpy as np

#make a list of dates to aligns with the ASO observations (they go as early as Jan-29 and as far out as the July-17)
years = np.arange(2013,2020,1)
start_month_day = '10-01'
end_month_day = '08-31'
#datelist = get_InSitu_obs.make_dates(years, start_month_day, end_month_day, WY = True)

# observations 
get_InSitu_obs.Get_Monitoring_Data_Threaded_dp(years, start_month_day, end_month_day, WY = True)

#combine years
get_InSitu_obs.combine_dfs(years)

# Code for generating ML dataframe using nearest in situ monitoring sites

In [ ]:
import GeoDF
output_res = 300

# GeoDF used to create a dataframe for ML model development. Its function is to connect in situ observations to gridded locations
for region in region_list:
    path = f"{HOME}/SWEMLv2.0/data/ASO/{region}/{output_res}M_SWE_parquet"
    if os.path.isdir(path) == True:
        print(region)
        #load snotel meta location data, use haversive function
        GeoDF.fetch_snotel_sites_for_cellids(region, output_res) # Using known up to date sites

        # Get geophysical attributes for each site, need to see how to add output resolution
        gdf = GeoDF.GeoSpatial(region, output_res)

        #use geodataframe with lat/long meta of all sites to determine slope, aspect, and elevation
        metadf = GeoDF.extract_terrain_data_threaded(gdf, region, output_res)
    else:
        print(f"No ASO data for {region}")




## Connect Snotel to each ASO obs

- change nearest_sites name to ns

In [ ]:
import Obs_to_DF
output_res = 300

#Connect nearest snotel observations with ASO data, makes a parquet file for each date  -  test to see if this works - need to just load the SNOTEL file, not collect them as in the function
for region in region_list:
    path = f"{HOME}/SWEMLv2.0/data/ASO/{region}/{output_res}M_SWE_parquet"
    if os.path.isdir(path) == True:
        print(region)
        dates = []
        manual = False
        Obs_to_DF.Nearest_Snotel_2_obs_MultiProcess(region, output_res, manual, dates) 
    else:
        print(f"No ASO data for {region}")


In [ ]:
import GeoDF

output_res = 300

#Connect cell ids with ASO obs and snotel obs to geospatial features
for region in region_list:
    path = f"{HOME}/SWEMLv2.0/data/ASO/{region}/{output_res}M_SWE_parquet"
    if os.path.isdir(path) == True:
        print(region)
        GeoDF.add_geospatial_threaded(region, output_res)
    else:
        print(f"No ASO data for {region}")

# Get NASA VIIRS fraction snow covered area for each location 

* Make sure the code grabs all dates for each region


In [ ]:
import get_VIIRS_SCA
output_res = 300
threshold = 20

#check to see if the VIIRS data is available locally, if not, get from CIROH AWS - I think all of this data is for the incorrect year...
#get_VIIRS_SCA.get_VIIRS_from_AWS()

#Connect VIIRS data to dataframes
for region in region_list:
    path = f"{HOME}/SWEMLv2.0/data/ASO/{region}/{output_res}M_SWE_parquet"
    if os.path.isdir(path) == True:
        print(region)
        get_VIIRS_SCA.augment_SCA_mutliprocessing(region, output_res, threshold)
    else:
        print(f"No ASO data for {region}")


In [ ]:
import get_Precip

'''
note*, if using python > 3.9, you will likely need to change the ee package to from io import StringIO
'''

import os
HOME = os.path.expanduser('~')

#gets precipitation for each location, accumulates it through the water year

#set start/end date for a water year
years = [2013, 2014, 2015, 2016, 2017, 2018, 2019]
output_res = 300
threshold = 20

for region in region_list:
    path = f"{HOME}/SWEMLv2.0/data/ASO/{region}/{output_res}M_SWE_parquet"
    if os.path.isdir(path) == True:
        print(region)
        get_Precip.get_precip_threaded(region, output_res, years)
    else:
        print(f"No ASO data for {region}")

    #Connect precipitation to processed DFs
    get_Precip.Make_Precip_DF(region, output_res, threshold)


In [ ]:
import get_Seasonality

output_res = 300
threshold = 20
for region in region_list:
    #process snotel sites to make "snow hydrograph features" to determine above/below average WY conditions
    get_Seasonality.seasonal_snotel()


    #get the Day of season metric for each dataframe
    get_Seasonality.add_Seasonality(region, output_res, threshold)

In [62]:
import sturm_processer as stpro

# output_res = 300
# threshold = 20
# for region in region_list:
#     get_Sturm.add_Sturm(region, output_res, threshold)


for region in region_list:
    input_directory = f"{HOME}/SWEMLv2.0/data/TrainingDFs/{region}/300M_Resolution/Seasonality_PrecipVIIRSGeoObsDFs/20_fSCA_Thresh"
    sturm_file = f"{HOME}/SWEMLv2.0/data/SnowClassification/SnowClass_NA_300m_10.0arcsec_2021_v01.0.tif" #https://nsidc.org/data/nsidc-0768/versions/1
    output_directory = f"{HOME}/SWEMLv2.0/data/TrainingDFs/{region}/300M_Resolution/Sturm_Seasonality_PrecipVIIRSGeoObsDFs/20_fSCA_Thresh"
    
    # Create output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)
    
    stpro.process_sturm_data_for_files(input_directory, sturm_file, output_directory)

Sturm file bounds: BoundingBox(left=-180.0, bottom=0.0, right=-10.0, top=90.0)


Processing Parquet Files:  50%|█████     | 1/2 [00:04<00:04,  4.65s/it]

                              cell_id       Date  cen_lat  cen_lon  \
79625  Northwest_300M_47.536_-123.525 2016-03-29   47.536 -123.525   
6863   Northwest_300M_48.041_-123.832 2016-03-29   48.041 -123.832   
57788  Northwest_300M_47.687_-123.967 2016-03-29   47.687 -123.967   
48990  Northwest_300M_47.749_-123.627 2016-03-29   47.749 -123.627   
84441   Northwest_300M_47.501_-124.07 2016-03-29   47.501 -124.070   

       Elevation_m  Slope_Deg  Aspect_Deg     swe_m   ns_1   ns_2  ...  \
79625         1188         44         320  0.043129  157.2   88.1  ...   
6863            43         16         225  0.000205  111.8  157.2  ...   
57788          922         25         324  0.000000  157.2  111.8  ...   
48990          735         49         106  1.842366  157.2  111.8  ...   
84441          557         27          37  0.000000  157.2  111.8  ...   

       ns_5_week_mean  ns_6_week_mean  Seasonal_ns_1_rel  Seasonal_ns_2_rel  \
79625        1.712245       30.708163           1.35267

Processing Parquet Files: 100%|██████████| 2/2 [00:06<00:00,  3.40s/it]


                              cell_id       Date  cen_lat  cen_lon  \
58369  Northwest_300M_47.685_-123.446 2016-02-08   47.685 -123.446   
2922   Northwest_300M_48.068_-123.916 2016-02-08   48.068 -123.916   
53207   Northwest_300M_47.72_-123.765 2016-02-08   47.720 -123.765   
81612  Northwest_300M_47.523_-123.371 2016-02-08   47.523 -123.371   
77825   Northwest_300M_47.55_-123.174 2016-02-08   47.550 -123.174   

       Elevation_m  Slope_Deg  Aspect_Deg     swe_m  ns_1  ns_2  ...  \
58369         1331         38         245  0.123879  93.0  72.9  ...   
2922           329         12         258  0.000000  72.9  93.0  ...   
53207          940         41         206  0.000000  93.0  72.9  ...   
81612          743         53         147  0.000000  93.0  56.1  ...   
77825          399          2         180  0.000691  56.1  93.0  ...   

       ns_5_week_mean  ns_6_week_mean  Seasonal_ns_1_rel  Seasonal_ns_2_rel  \
58369       44.640816        2.895918           1.160310           

...

                              cell_id       Date  cen_lat  cen_lon  \
4998    Southwest_300M_37.588_-119.17 2019-06-14   37.588 -119.170   
15798  Southwest_300M_37.435_-119.383 2019-06-14   37.435 -119.383   
12642   Southwest_300M_37.48_-118.873 2019-06-14   37.480 -118.873   
1227    Southwest_300M_37.68_-119.192 2019-06-14   37.680 -119.192   
11799  Southwest_300M_37.489_-119.286 2019-06-14   37.489 -119.286   

       Elevation_m  Slope_Deg  Aspect_Deg     swe_m    ns_1  ns_2  ...  \
4998          2658         39         226  0.308854     1.5   0.0  ...   
15798         2555         18         109  0.000000     0.6  16.2  ...   
12642         2100         24         358  0.768744    50.7   2.1  ...   
1227          3047         17         183  3.272123 -9999.0   0.0  ...   
11799         3209         39         266  0.000000     0.6   1.5  ...   

       ns_5_week_mean  ns_6_week_mean  Seasonal_ns_1_rel  Seasonal_ns_2_rel  \
4998       147.344898     -182.002041          -0.00740

Processing Parquet Files:  99%|█████████▉| 98/99 [01:27<00:00,  1.12it/s]

                            cell_id       Date  cen_lat  cen_lon  Elevation_m  \
381   Southwest_300M_37.562_-118.97 2017-01-28   37.562 -118.970         1945   
124    Southwest_300M_37.6_-119.029 2017-01-28   37.600 -119.029         1987   
170   Southwest_300M_37.595_-119.01 2017-01-28   37.595 -119.010         1945   
156  Southwest_300M_37.598_-118.997 2017-01-28   37.598 -118.997         1945   
178  Southwest_300M_37.595_-118.989 2017-01-28   37.595 -118.989         1945   

     Slope_Deg  Aspect_Deg     swe_m   ns_1  ns_2  ...  ns_5_week_mean  \
381          0           0  1.501426  104.2  96.1  ...    -1392.579592   
124          5         264  1.571528  104.2  96.1  ...       27.706122   
170          0           0  0.847414  104.2  96.1  ...       37.355102   
156          0           0  0.523601  104.2  96.1  ...       37.355102   
178          0           0  0.585433  104.2  96.1  ...       37.355102   

     ns_6_week_mean  Seasonal_ns_1_rel  Seasonal_ns_2_rel  Seasonal_

Processing Parquet Files: 100%|██████████| 99/99 [01:28<00:00,  1.12it/s]

                              cell_id       Date  cen_lat  cen_lon  \
8237   Southwest_300M_37.273_-118.941 2019-07-14   37.273 -118.941   
7732   Southwest_300M_37.283_-118.797 2019-07-14   37.283 -118.797   
7903    Southwest_300M_37.281_-118.73 2019-07-14   37.281 -118.730   
3132   Southwest_300M_37.383_-118.838 2019-07-14   37.383 -118.838   
10109  Southwest_300M_37.235_-118.725 2019-07-14   37.235 -118.725   

       Elevation_m  Slope_Deg  Aspect_Deg     swe_m  ns_1    ns_2  ...  \
8237          1945          0           0  0.000000   4.8     0.0  ...   
7732          2307         16         342  0.188220   0.0     4.8  ...   
7903          2158          9         264  0.308242   0.0     2.7  ...   
3132          2296          6         279  0.364586   0.0     2.7  ...   
10109         2184          4         333  0.480882   1.0 -9999.0  ...   

       ns_5_week_mean  ns_6_week_mean  Seasonal_ns_1_rel  Seasonal_ns_2_rel  \
8237     -2856.524490    -5711.759184           1.55864

In [ ]:
import vegetation_processer as vegpro

HOME = os.path.expanduser('~')
region_list = ['Northwest', 'SouthernRockies', 'Southwest']
for region in region_list:
    input_directory = f"{HOME}/SWEMLv2.0/data/TrainingDFs/{region}/300M_Resolution/Sturm_Seasonality_PrecipVIIRSGeoObsDFs/20_fSCA_Thresh"
    vegetation_file = f"{HOME}/SWEMLv2.0/USA_NALCMS_landcover_2020_120m.tif"
    output_directory = f"{HOME}/SWEMLv2.0/data/TrainingDFs/{region}/300M_Resolution/Vegetation_Sturm_Seasonality_PrecipVIIRSGeoObsDFs/20_fSCA_Thresh"
    
    # Create output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)
    
    vegpro.process_vegetation_data_for_files(input_directory, vegetation_file, output_directory)

## Next steps
* Explore why errors in precip sites above
* add in situ obs - seasonality based on the historical neareste x monitoring stations - like a historical average to-date swe value unit hydrograph based on the day of year? This will include a historical time of year of normal swe value and a swe value of year compared to normal
* albedo metric
